# XGboost model

In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')


In [2]:
anomoly  = False
# load data either by anomoly or not
if anomoly:
    X=pd.read_csv("https://raw.githubusercontent.com/KelvinYQC/msia420PA_project/main/Data/with_anomaly.csv")
else:
    X=pd.read_csv("https://raw.githubusercontent.com/KelvinYQC/msia420PA_project/main/Data/without_anomaly.csv")
y = X['booking_status']
X.drop(['booking_status'], axis = 1, inplace = True)


In [3]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size=None, train_size=0.3)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV

pipeline = Pipeline(steps=[("scaler", StandardScaler()), ("clf", xgb.XGBClassifier())])


parameters = {
    'clf__learning_rate': [0.001, 0.01, 0.05, 0.1, None],
    'clf__max_depth': [2,5,7,10,15,30, None]
}

clf = HalvingRandomSearchCV(
    pipeline, parameters, scoring="roc_auc", n_jobs=-1, factor=4, cv = 10
)

# clf = RandomizedSearchCV(estimator=pipeline,
#                          param_distributions = parameters,
#                          n_iter=20,
#                          verbose=1,
#                          cv = 5,
#                          scoring = 'roc_auc')
clf_xgb = clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)


Best parameters: {'clf__max_depth': 15, 'clf__learning_rate': None}


In [8]:
from sklearn.metrics import roc_auc_score

y_pred = clf_xgb.predict(X_test)
classifier_score_xgb = roc_auc_score(y_test, y_pred)

print("The resulting roc auc score: ")
print(round(classifier_score_xgb,3))

The resulting roc auc score: 
0.854


The AUC from XGboost model is 0.854.